<a href="https://colab.research.google.com/github/johnjaejunlee/John-Lee/blob/master/Practice%20of%20Classifcation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
import numpy as np
import pandas as pd
import sklearn.linear_model as lm
import sklearn.svm as svm
import sklearn.model_selection as mod_sel
import sklearn.metrics as met
from sklearn.manifold import TSNE
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from sklearn.datasets import load_digits
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from matplotlib.pyplot import colormaps as cm

In [ ]:
## digit 일부 발췌 & 확인
digits = load_digits()
fig, ax_array = plt.subplots(7, 7)
axes = ax_array.flatten()
for i, ax in enumerate(axes):
    ax.imshow(digits.images[i], cmap='gray_r')
plt.setp(axes, xticks=[], yticks=[], frame_on=False)
plt.tight_layout(h_pad=1, w_pad=1)

In [ ]:
## X, y 설정 
digits = load_digits()
X = digits.data
y = digits.target
label = list(set(y))

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

images = digits.images
images
## Divide data for traning (70%) and test (30%)
X_train, X_test, y_train, y_test = mod_sel.train_test_split(X, y, test_size=0.3, random_state=5)

## Fit SVM to the data, X_train and y_train
classifier = LogisticRegression(penalty= 'l2', C= 100, multi_class= 'multinomial', solver= 'lbfgs', max_iter= 10000, fit_intercept= True)
classifier.fit(X_train,y_train)

## Performance test
y_pred_test = classifier.predict(X_test)
accuracy_test = met.accuracy_score(y_test, y_pred_test)

y_pred_train = classifier.predict(X_train)
accuracy_train = met.accuracy_score(y_train, y_pred_train)

print("Accuracy(Naive):tested=", accuracy_test, "trained=", accuracy_train )

## Cross Entropy (Loss Function)
y_prob = classifier.predict_proba(X_test) 
y_prob_train = classifier.predict_proba(X_train)
print("Cross Entropy: tested=", log_loss(y_test, y_prob), "trained=", log_loss(y_train, y_prob_train))

## Evaluation: L-Fold Cross Validation
y_pred = mod_sel.cross_val_predict(classifier, X, y, cv=5, n_jobs=4)
c_mat = met.confusion_matrix(y, y_pred)
###############################################################################
import seaborn as sns
print("\n 교차검증 후 실제 final_accuracy (반올림 x):", 100*met.accuracy_score(y,y_pred), "%\n")
print(classification_report(y, y_pred))
print("\nConfusion matrix")
print(c_mat)
print("\n\n")
plt.figure(figsize=(15,15), frameon=True, facecolor='white', dpi=225)
sns.heatmap(c_mat, fmt="d", annot=True, linecolor='white', linewidths=3)
plt.title("Confusion Matrix", fontsize = 30)
plt.rcParams.update({'font.size': 15})
plt.ylabel('Actual Value',fontsize = 20)
plt.xlabel('Prediction Value', fontsize=20)
plt.savefig('confusionmatrix.png')
plt.show()
###############################################################################

In [ ]:
## Checking Estimated Probability of Dataset (Partial Check)
y_prob = classifier.predict_proba(X)
print('Classes: ', classifier.classes_)
print( 'Original/Predicted label: ', y[1], y_pred[1] )
print( 'Estimated probabilities: ', y_prob[1,:]) ## sigmoid function

In [ ]:
## 결과 확인
f=plt.figure(figsize=(16,12), facecolor='lightgreen', dpi=None)

for i, ax in enumerate(axes):
    image = images[i]
    sub =f.add_subplot(7, 7, i+1)
    sub.imshow(image, cmap='GnBu_r')
    plt.xticks([])
    plt.yticks([])
    plt.rcParams.update({'font.size': 15})
    sub.set_title('predict: %i\n actual: %i' % (y_pred[i], y[i]))
    plt.subplots_adjust(hspace =1.2,)

f.suptitle('Compare Prediction and Targets Value', fontsize = 30)
plt.show()

In [ ]:
###### 학습된 prediction과 실제 y test값 비교
up_count = 0
down_count = 0
for i in range(axes.size):
    if y_pred[i]==y[i]: 
        up_count = up_count+1
    else: down_count = down_count +1  ##y_pred 값과 y_test 값을 비교해서 같으면 up, 다르면 down
print (up_count, down_count, up_count + down_count, up_count/(up_count + down_count)) ##Data가 적기 때문에 accuracy 조금 낮음

In [ ]:
## 최종 각 class 별 intercept & theta 값 확인 
print( "theta_0 (intercept) = ") 
print(classifier.intercept_) ## class 별 intercept 값
print( "theta[1:M] = " )
print(classifier.coef_) ## class별 theta 값

In [ ]:
## ROC_AUC (open source 활용)
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle

from sklearn import svm
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp

# Binarize the output
y_bin = label_binarize(y, classes=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
n_classes = 10

X = np.c_[X, random_state.randn(n_samples, 200 * n_features)]

X_train, X_test, y_train, y_test = train_test_split(X, y_bin, test_size=.3)
# Learn to predict each class against the other
classifier = OneVsRestClassifier(svm.SVC(kernel='linear', C=0.01, probability=True, gamma='auto'))
y_score = classifier.fit(X_train, y_train).decision_function(X_test)

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["macro"], tpr["macro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

plt.figure()
lw = 2
plt.plot(fpr[2], tpr[2], color='blue', lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[2])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()